In [21]:
import pandas as pd
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from torch import optim
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
torch.__version__

'1.9.1.post3'

In [22]:
democrats = pd.read_parquet("../10_datasets/democrats")
republicans = pd.read_parquet("../10_datasets/neutral")
neutral = pd.read_parquet("../10_datasets/republican.parquet")

In [23]:
democrats.shape

(4937, 7)

In [24]:
republicans["subreddit"].value_counts()

Republican         3419
NeutralPolitics    1341
Name: subreddit, dtype: int64

In [25]:
republicans.shape

(4760, 7)

In [26]:
neutral.shape

(4809, 7)

In [27]:
df = pd.concat([democrats, republicans, neutral])

In [28]:
df.head()

,id,total_post,subreddit,score,type,title,text
index,,,,,,,
77764,ffqkhbh,There's no such thing as free healthcare. Some...,democrats,1.0,comment,None,None
65534,fkt7z5l,Figures about the deployment of tests ? Trump'...,democrats,1.0,comment,None,None
25536,g78fjn4,"Money. Personally, I believe the Republicans h...",democrats,1.0,comment,None,None
31018,g3vpj3c,I have family members who get their hair cut.\...,democrats,1.0,comment,None,None
13123,gbje5mg,Stuttering doesn’t make someone a bad person. ...,democrats,1.0,comment,None,None


In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(df["subreddit"])
df['encoded_subreddit'] = le.transform(df['subreddit'])

In [30]:
df['subreddit'].value_counts()

Republican         8228
democrats          4937
NeutralPolitics    1341
Name: subreddit, dtype: int64